In [ ]:
!pip install mido
from music21 import converter, instrument, note, chord, duration
import numpy as np
from music21 import converter, instrument, note, chord, stream
from collections import OrderedDict
import mido
import numpy as np
from collections import Counter
import math
from music21 import converter, pitch

Pitch Class Histogram

In [ ]:
def tokenize_midi_file(midi_file):
    # Load the MIDI file
    mid = mido.MidiFile(midi_file)

    # Initialize the pitch class histogram
    pitch_classes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    pitch_histogram = np.zeros(12)

    # Iterate over all tracks and messages in the MIDI file
    for track in mid.tracks:
        for msg in track:
            if msg.type == 'note_on':
                # Get the pitch of the note
                pitch = msg.note % 12

                # Update the pitch class histogram
                pitch_histogram[pitch] += 1

    # Normalize the pitch class histogram
    total_count = np.sum(pitch_histogram)
    pitch_histogram /= total_count

    # Calculate the entropy
    entropy = -np.sum(pitch_histogram * np.log2(pitch_histogram + 1e-10))

    return pitch_histogram, entropy

def print_pitch_histogram(pitch_histogram):
    pitch_classes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    print("Pitch Class Histogram:")
    for pitch, count in enumerate(pitch_histogram):
        pitch_name = pitch_classes[pitch]
        print(f"{pitch_name}: {count}")

# Example usage
midi_file = '/content/Jazz RNN w: Seed 8 for paper.mid'
pitch_histogram, entropy = tokenize_midi_file(midi_file)
print_pitch_histogram(pitch_histogram)
print("Entropy:", entropy)

In [ ]:
import matplotlib.pyplot as plt

def plot_pitch_histogram(pitch_histogram):
    pitch_classes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    x = range(len(pitch_histogram))

    plt.bar(x, pitch_histogram)
    plt.xlabel('Pitch Class')
    plt.ylabel('Frequency')


    # Set the x-axis tick labels to pitch class names
    plt.xticks(x, pitch_classes)

    plt.show()

# Example usage
midi_file = '/content/Jazz RNN w: Seed 8 for paper.mid'
pitch_histogram, entropy = tokenize_midi_file(midi_file)
plot_pitch_histogram(pitch_histogram)

Grooving Pattern

In [ ]:
def calculate_grooving_pattern(bar_notes):
    grooving_pattern = np.zeros(64)  # Initialize the grooving pattern as a 64-dimensional vector
    for note_obj in bar_notes:
        if isinstance(note_obj, note.Note):
            position = int(note_obj.offset * 4)  # Calculate the position in the bar
            if position < 64:  # Ensure the position is within the range of the grooving pattern
                grooving_pattern[position] = 1
    return grooving_pattern.astype(int)


def calculate_grooving_pattern_similarity(grooving_pattern_a, grooving_pattern_b):
    q = len(grooving_pattern_a)  # Dimensionality of the grooving pattern vectors
    xor_result = np.bitwise_xor(grooving_pattern_a, grooving_pattern_b)  # Apply the XOR operation
    similarity = 1 - (1/q) * np.sum(xor_result)  # Calculate the similarity score using the updated formula
    return similarity


# Load the generated music as a MIDI file
generated_music = converter.parse("/content/Jazz RNN w: Seed 8 for paper.mid")

# Initialize a list to store the generated grooving patterns
generated_grooving_patterns = []

# Iterate over bars in the generated music
for bar in generated_music.parts[0].measures(1, None):
    # Extract notes from the current bar
    bar_notes = [note_obj for note_obj in bar.notesAndRests if isinstance(note_obj, (note.Note, note.Rest))]

    # Calculate the grooving pattern for the current bar
    current_bar_grooving_pattern = calculate_grooving_pattern(bar_notes)

    # Append the grooving pattern to the list
    generated_grooving_patterns.append(current_bar_grooving_pattern)

# Calculate the grooving pattern similarity for each pair of bars
grooving_pattern_similarities = []
for i in range(len(generated_grooving_patterns) - 1):
    similarity = calculate_grooving_pattern_similarity(generated_grooving_patterns[i], generated_grooving_patterns[i+1])
    grooving_pattern_similarities.append(similarity)

# Calculate the average pattern similarity
average_similarity = np.mean(grooving_pattern_similarities)


print("Grooving Pattern Similarities:")
for i, similarity in enumerate(grooving_pattern_similarities):
    print(f"Bar {i+1} to Bar {i+2}: {similarity}")
print(f"Average Pattern Similarity: {average_similarity}")

import matplotlib.pyplot as plt

# Plot the grooving pattern similarity
plt.figure(figsize=(8, 4))
x = range(1, len(grooving_pattern_similarities) + 1)  # X-axis: Bar pairs
y = grooving_pattern_similarities  # Y-axis: Similarity scores
plt.plot(x, y, marker='o', linestyle='-', color='blue')
plt.xlabel('Bar Pair')
plt.ylabel('Similarity Score')

plt.grid(True)
plt.show()